In [31]:
# Implementing freeze-thaw bayesian optimization with two-level Gaussian processes
# A global GP models the asymptotic mean of the learning curves of each HP-config
# Local GPs model the learning curves of each HP-config

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, Kernel, Sum, WhiteKernel
import numpy as np
import matplotlib.pyplot as plt
from ft_testfunction import global_xD, local_xD
import scipy as sc

# Hyper-Hyperparameters of Freeze-Thaw Bayesian Optimization
ALPHA,BETA = 1,0.5
NOISE = 0.1
B_OLD=2 # 10
B_NEW=2 # 3
MAX_EPOCHS=100
EPOCH_STEP=1
N_SAMPLES_MC=10 # number of samples for Monte Carlo integration
N_FANT=5 # number of observations we fantasize for each point in the basket
N_INIT_CONFIGS=10 # number of random initializations for the optimization
N_INIT_EPOCHS=5 # number of epochs trained for initial configs
INFERRED_MEAN = 0.8 # inferred mean of the global GP
MATER_NU = 2.5 # Matern kernel parameter

# Meta-Parameters of the task
OBS_MIN = 0 # minimal loss value
OBS_MAX = 1 # maximal loss value


In [32]:
'''
Equation 3: Expected Improvement-formula
Equation 4: Entropy Search-formula
Equation 5&6: Exponential Decay-kernel for Gaussian Process
Equation 12,13,17 & 18: Posterior distribution of global Gaussian Process
Equation 14 & 19: Posterior predictive distribution of global Gaussian Process
Equation 15 & 20: Posterior predictive distribution of a local Gaussian Process with existing oberservations of this HP-config
Equation 16 & 21: Posterior predicitve distribution of a local Gaussian Process without existing oberservations of this HP-config
'''


'\nEquation 3: Expected Improvement-formula\nEquation 4: Entropy Search-formula\nEquation 5&6: Exponential Decay-kernel for Gaussian Process\nEquation 12,13,17 & 18: Posterior distribution of global Gaussian Process\nEquation 14 & 19: Posterior predictive distribution of global Gaussian Process\nEquation 15 & 20: Posterior predictive distribution of a local Gaussian Process with existing oberservations of this HP-config\nEquation 16 & 21: Posterior predicitve distribution of a local Gaussian Process without existing oberservations of this HP-config\n'

In [33]:
# Definition of exponential-decay kernel for local GPs
class ExponentialDecayNoiseKernel(Kernel):
    def __init__(self, alpha=1.0,beta=0.5,noise=0.1):
        self.beta=beta
        self.alpha=alpha
        self.noise = noise
    def __call__(self, X, Y=None,eval_gradient=False):
        if Y is None:
            Y = X
        # if eval_gradient:
        #     return (self.beta**self.alpha)/((X.flatten()[:,None]+Y.flatten()[None,:])+self.beta)**self.alpha,np.identity(X.shape[0])
        X=np.array(X)
        Y=np.array(Y)
        return ((self.beta**self.alpha)/((X[None,:]+Y[:,None])+self.beta)**self.alpha + self.noise*np.where(X[None,:]-Y[:,None] == 0, 1, 0)).T
    def diag(self, X):
        return np.diag(self(X))
    def is_stationary(self):
        return False

In [34]:
# Start by observing N_INIT_CONFIGS random configurations for N_INIT_EPOCHS epoch each
bounds={'HP1':(1.,5.)}#,'HP2':(1.,5.)}
observed_configs_dicts={}
observed_configs_list=[]
for i in range(N_INIT_CONFIGS):
    new_config=np.empty(0)
    for key in bounds.keys():
        new_config=np.append(new_config,np.round(np.random.uniform(bounds[key][0],bounds[key][1]),2))
    
    # Observe the new configuration for N_INIT_EPOCHS epochs
    f_space = np.linspace(1,N_INIT_EPOCHS,N_INIT_EPOCHS)
    experimental_data=local_xD(np.array(new_config),f_space,noise=0.01)
    observed_configs_dicts['_'.join([str(config) for config in new_config])]=(f_space,experimental_data)


    observed_configs_list.append(new_config)
observed_configs_list=np.array(observed_configs_list)



print(observed_configs_dicts)
print(observed_configs_list)



{'3.13': (array([1., 2., 3., 4., 5.]), array([0.93, 0.85, 0.82, 0.75, 0.69])), '1.63': (array([1., 2., 3., 4., 5.]), array([0.94, 0.89, 0.82, 0.78, 0.73])), '4.05': (array([1., 2., 3., 4., 5.]), array([0.95, 0.89, 0.83, 0.75, 0.74])), '1.97': (array([1., 2., 3., 4., 5.]), array([0.95, 0.88, 0.83, 0.79, 0.74])), '3.76': (array([1., 2., 3., 4., 5.]), array([0.92, 0.88, 0.81, 0.78, 0.7 ])), '2.73': (array([1., 2., 3., 4., 5.]), array([0.94, 0.86, 0.8 , 0.76, 0.69])), '3.79': (array([1., 2., 3., 4., 5.]), array([0.94, 0.86, 0.79, 0.76, 0.7 ])), '1.8': (array([1., 2., 3., 4., 5.]), array([0.94, 0.89, 0.82, 0.76, 0.75])), '3.71': (array([1., 2., 3., 4., 5.]), array([0.92, 0.87, 0.8 , 0.74, 0.71])), '3.64': (array([1., 2., 3., 4., 5.]), array([0.93, 0.88, 0.84, 0.78, 0.71]))}
[[3.13]
 [1.63]
 [4.05]
 [1.97]
 [3.76]
 [2.73]
 [3.79]
 [1.8 ]
 [3.71]
 [3.64]]


In [35]:
# from botorch.models import SingleTaskGP
# from gpytorch.kernels import MaternKernel, ScaleKernel
# from gpytorch.means import ConstantMean
# import torch
#MaternKernel(nu=2.5)
# print(torch.from_numpy(observed_configs_list))


# Define the kernel for the global GP
kernel_global = Matern(nu=MATER_NU)
kernel_local = ExponentialDecayNoiseKernel(alpha=ALPHA,beta=BETA,noise=NOISE)

def compute_k_x(x,new_x,kernel):
    k_x = kernel_global(x)
    k_x_star = kernel_global(x,new_x)
    k_star_star = kernel_global(new_x)
    return k_x,k_x_star,k_star_star

def compute_k_t_n(x,new_x,kernel):
    k_t=kernel(x)
    k_t_star=kernel(x,new_x)
    k_star_star=kernel(new_x)
    return k_t,k_t_star,k_star_star

def compute_k_t(k_ts):
    return sc.linalg.block_diag(*k_ts)

def compute_o(config_list,config_dict):
    o=[]
    for config in config_list:
        observations=config_dict['_'.join([str(c) for c in config])][1]
        o=sc.linalg.block_diag(*([o,np.ones((len(observations),1))]))
    return o[1:]

def compute_lambda(o,k_t_inv):
    return o.T@k_t_inv@o

def compute_gamma(o,k_t_inv,y,m):
    return o.T@k_t_inv@(y-o@m)

def constant_mean(x):
    return INFERRED_MEAN*np.ones(x.shape[0])

def compute_y_vector(config_list,config_dict):
    y_vec=np.empty(0)
    for config in config_list:
        observations=config_dict['_'.join([str(c) for c in config])][1]
        y_vec=np.append(y_vec,observations)
    return y_vec

def compute_c(k_x_inv,lambd):
    return np.linalg.inv(k_x_inv+lambd)

def compute_omega(k_t_n_star,k_t_n_inv):
    obs_steps=k_t_n_inv.shape[0]
    predict_steps=k_t_n_star.shape[1]
    return np.ones(predict_steps)-k_t_n_star.T@k_t_n_inv@np.ones(obs_steps)

def compute_mu(m,c,gamma):
    return (m+c@gamma)

# Equation 14/19:
def compute_mu_x_star(m,k_x_star,k_x_inv,my,means_vec):
    return m+k_x_star.T@k_x_inv@(my-means_vec)
def compute_sigma_x_star_star(k_x_star_star,k_x_star,k_x,lambd_inv):
    return k_x_star_star-k_x_star.T@np.linalg.inv(k_x+lambd_inv)@k_x_star

# Equation 16/21:
def compute_sigma_n_star_new(k_t_star_star,sigma_star_star):
    return k_t_star_star-sigma_star_star

# Equation 15/20:
def compute_mu_n_star_ex(k_t_n_star,k_t_n_inv,y_n,omega_n,mu_n):
    return k_t_n_star.T@k_t_n_inv@y_n+(omega_n*mu_n)
def compute_sigma_n_star_ex(k_t_n_star_star,k_t_n_star,k_t_n_inv,omega_n,c_nn):
    return k_t_n_star_star-k_t_n_star.T@k_t_n_inv@k_t_n_star+omega_n@(c_nn*omega_n.T)

In [43]:
new_x = np.array([[2.69],[3],[1]]) # Placeholer
print(observed_configs_list)
# print(new_x)
k_x,k_x_star,k_x_star_star = compute_k_x(observed_configs_list,new_x,kernel_global)
k_x_inv = np.linalg.inv(k_x)
# print(f"Kx:\n{k_x}\n Kx*:\n{k_x_star}\n Kx**:\n{k_x_star_star}")

new_x_n=np.array([6,10,50,100]) # Placeholer
curve=observed_configs_dicts["_".join([str(x) for x in observed_configs_list[0]])]
# print(curve)
# print(new_x_n)
k_t_n,k_t_n_star,k_t_n_star_star=compute_k_t_n(curve[0],new_x_n,kernel_local)
k_t_n_inv=np.linalg.inv(k_t_n)
# print(f"Ktn:\n{k_t_n}\n Ktn*:\n{k_t_n_star}\n Ktn**:\n{k_t_n_star_star}")

k_t=compute_k_t([k_t_n]*len(observed_configs_list)) # Placeholer
k_t_inv=np.linalg.inv(k_t)
# print(f"Kt:\n {k_t}")

o = compute_o(observed_configs_list,observed_configs_dicts)
# print(f"O:\n{o}")

lambd = compute_lambda(o,k_t_inv)
lambd_inv = np.linalg.inv(lambd)
# print(f"Lambda:\n{lambd}")

means_vec = constant_mean(observed_configs_list)
# print(f"Means:\n{means_vec}")

y_vec = compute_y_vector(observed_configs_list,observed_configs_dicts)
# print(f"Y:\n{y_vec}")

gamma = compute_gamma(o,k_t_inv,y_vec,means_vec)
# print(f"Gamma:\n{gamma}")

c = compute_c(k_x_inv,lambd)
# print(f"C:\n{c}")

mu_global = compute_mu(means_vec,c,gamma)
# print(f"Mu:\n{mu_global}")

omega_n = compute_omega(k_t_n_star,k_t_n_inv)
# print(f"Omega n:\n{omega_n}")

mu_x_star = compute_mu_x_star(constant_mean(new_x),k_x_star,k_x_inv,mu_global,means_vec)
print(f"μx*:\n{mu_x_star}")

sigma_x_star_star = compute_sigma_x_star_star(k_x_star_star,k_x_star,k_x,lambd_inv)
print(f"Σx**:\n{sigma_x_star_star}")

mu_n_star_ex = compute_mu_n_star_ex(k_t_n_star,k_t_n_inv,curve[1],omega_n,mu_global[0])
print(f"μn* (existing):\n{mu_n_star_ex}")

sigma_n_star_ex = compute_sigma_n_star_ex(k_t_n_star_star,k_t_n_star,k_t_n_inv,omega_n,c[0,0])
print(f"Σn* (existing):\n{sigma_n_star_ex}")

[[3.13]
 [1.63]
 [4.05]
 [1.97]
 [3.76]
 [2.73]
 [3.79]
 [1.8 ]
 [3.71]
 [3.64]]
μx*:
[0.75582842 0.75156438 0.78687918]
Σx**:
[[ 6.15892616e-02  3.41423603e-02 -1.03909138e-02]
 [ 3.41423603e-02  4.89731504e-02  4.32966063e-04]
 [-1.03909138e-02  4.32966063e-04  4.55371490e-01]]
μn* (existing):
[0.77268138 0.76190711 0.75253797 0.7521358 ]
Σn* (existing):
[[0.22598291 0.12448997 0.11752693 0.11575169]
 [0.12448997 0.22423355 0.11833211 0.11629813]
 [0.11752693 0.11833211 0.21765606 0.11645047]
 [0.11575169 0.11629813 0.11645047 0.21585482]]


In [44]:

'''
Our Bayesian optimization strategy proceeds by maintaining a basket of B = Bold + Bnew candidate
models. Bold represents some number of models that have already been trained to some degree, while Bnew
represents some number of brand new models. In practice, we set Bold = 10 and Bnew = 3. The entire
basket is chosen using models with the maximum EI at the asymptote, which is computed using Equations
19 and 3. Each round, after a new observation has been collected, the basket is re-built using possibly
different models. This step is essentially standard Bayesian optimization using EI.
'''
# Fill the basket with configs
basket_new=[]
basket_old=[]


# Greedily choose the best HP-config using Equation 19 & 3 until B_OLD existing configs and B_NEW new configs are found
iterator=1.
while not (len(basket_new)==B_NEW):
    # Sample another config from EI and try to add it to the basket
    # If it is already in the basket, or the basket is full, skip it
    # TODO: Implement equations 19 & 3 here
    # Get the iterator best configs from EI
    sampled_EI_config = np.array([iterator]) # Take the iterator-th config from EI
    if not sampled_EI_config in observed_configs_list and not sampled_EI_config in basket_new and len(basket_new)<B_NEW:
        basket_new.append(sampled_EI_config)
    elif sampled_EI_config in observed_configs_list and not sampled_EI_config in basket_old and len(basket_old)<B_OLD:
        basket_old.append(sampled_EI_config)
    iterator+=0.2
if len(basket_old)<B_OLD:
    # If we did not find enough existing configs, fill the rest of the basket with random existing configs
    while len(basket_old)<B_OLD:
        random_config = observed_configs_list[np.random.randint(0,len(observed_configs_list))]
        if not random_config in basket_old:
            basket_old.append(random_config)

print(basket_new,basket_old)
a=np.zeros(len(basket_new+basket_old))


[array([1.]), array([1.2])] [array([2.73]), array([3.76])]


In [45]:
# Compute p_min over the basket using Monte Carlo simulation and Equation 19

# p_min is the posterior predictive distribution over the minimum of the global GP
# We will use it's entropy to observe configs with results that make the p_min distribution spikier

# Monte Carlo simulation
# Using the global GP, we create a discrete grid of the N_SAMPLES_MC highest EI configs
# From the grid, compute P_min and the entropy of P_min, H(P_min)

h_p_min=1

In [46]:
# For each config in the basket, N_FANT times fantasize an observation and recompute the information gain from it, collecting it in a
for k,config in enumerate(basket_new):
    for f_n in range(N_FANT):
        # Fantasize an observation using Equation 21
        # TODO fantasize an observation

        # Compute p_min_y, the new minimum distribution, including the fantasized observation y, again using Monte Carlo and Equation 19
        # TODO compute p_min_y

        # Compute the new entropy of p_min_y, H(P_min_y)
        # TODO compute H(P_min_y)
        h_p_min_y=0.5
        # 
        a[k]+=(h_p_min_y-h_p_min)/N_FANT

for k,config in enumerate(basket_old):
    for f_n in range(N_FANT):
        # Fantasize an observation using Equation 20
        # TODO fantasize an observation

        # Compute p_min_y, the new minimum distribution, including the fantasized observation y, again using Monte Carlo and Equation 19
        # TODO compute p_min_y

        # Compute the new entropy of p_min_y, H(P_min_y)
        # TODO compute H(P_min_y)
        h_p_min_y=0

        a[k+B_NEW]+=(h_p_min_y-h_p_min)/N_FANT


# Select the config with the highest information gain
best_config=(basket_new+basket_old)[np.argmax(a)]
print(best_config)


[1.]
